# Segentation of Lung CT scans

Data used in project can be downloaded via following link: http://medicaldecathlon.com/

## Intorduction

The idea of this project is to take input data in the form of a CT scan of the lungs, create and train a model which will do their segmentation and find abnormalities if they exist.

The typical use of convolutional neural networks is on classification tasks, where the output of an image is a single class label. In the problem that we are considering, we definitely should classify CT scans, but we also need to locate abnormalities if they exist. To do that, it is necessary to assign a label to each pixel of the CT scan. Of course, we are going to have two labels: normal and abnormal. All pixels that are classified as abnormal, together construct the abnormality of the considering lungs.

The model will be a convolutional neural network based on the U-net architecture.

#### What is U-net architecture and why we are going to use it?

The U-net architecture, the so-called "fully convolutional network," was designed in 2015. for the segmentation of biomedical images. U-Net has been successfully used in numerous studies and clinical applications for CT scan segmentation. 

It has a symetric structure in the shape of the letter U, and it consists of two main parts: the contracting path (coder) and the expansive path (decoder). This symmetric design of U-Net ensures a balanced extraction and reconstruction of features, leading to more accurate segmentation, which will be very useful considering the complex lung structures.

# ---------------------------------------------------------------------------------------------------------------

## Data Preproccessing

In [1]:
import os
import json
import numpy as np
import nibabel as nib
from matplotlib import pyplot as plt

In [2]:
def ct_to_slices(path):
    data = nib.load(path).get_fdata()
    [_, _, slices] = data.shape
    
    return [data[..., slice] for slice in range(slices)]

In [3]:
def convert_ct_dataset_to_slices(dataset_dir, output_dir):
    os.mkdir(output_dir)
    
    images_dir = os.path.join(output_dir, 'imagesTr')
    labels_dir = os.path.join(output_dir, 'labelsTr')
    os.mkdir(images_dir)
    os.mkdir(labels_dir)
    
    image_paths = None
    label_paths = None
    
    with open(os.path.join(dataset_dir, 'dataset.json'), 'r') as dataset_info:
        data = json.load(dataset_info)
        image_paths = [os.path.join(dataset_dir, scan['image']) for scan in data['training']]
        label_paths = [os.path.join(dataset_dir, scan['label']) for scan in data['training']]
        
    for (i, path) in enumerate(image_paths):
        for (j, slice) in enumerate(ct_to_slices(path)):
            plt.imsave(os.path.join(images_dir, f'{i + 1}_{j + 1}.png'), slice, cmap='gray')
    
    for (i, path) in enumerate(label_paths):
        for (j, slice) in enumerate(ct_to_slices(path)):
            plt.imsave(os.path.join(labels_dir, f'{i + 1}_{j + 1}.png'), slice, cmap='gray')

In [ ]:
convert_ct_dataset_to_slices('Task06_Lung', 'Task06_Lung_Sliced')

## Model creation  and training 

In [ ]:
 # TODO ...

## Model evaluation and testing

In [ ]:
# TODO ...

## References

1. U-Net: Convolutional Networks for Biomedical Image Segmentation: Olaf Ronneberger, Philipp Fischer, and Thomas Brox - Computer Science Department and BIOSS Centre for Biological Signalling Studies, University of Freiburg, Germany